In [2]:
# Import dependencies
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
from io import StringIO

# Initiate Variables

In [3]:
##### Initiate Global Variables
### API key for census website
MY_API_KEY = "xxx"

### Timeframes
CURRENT_YEAR = "2020"

### Geographic codes
# States
TEXAS = "48"
# Counties
HARRIS = "201"
FORTBEND = "157"
# Neighborhoods
ALL = "*"


In [4]:
# Get lists of zip codes
# Harris County
HarrisZips = pd.read_csv('definitions/HarrisZips.csv')
HarrisZips['zipCode'] = HarrisZips['zipCode'].astype('str')
HarrisZips = ",".join(HarrisZips['zipCode'])

# Ft Bend County
FtBendZips = pd.read_csv('definitions/FtBendZips.csv')
FtBendZips['zipCode'] = FtBendZips['zipCode'].astype('str')
FtBendZips = ",".join(FtBendZips['zipCode'])

# Define Functions

In [32]:
def buildTableCodes(variables_toquery, entireTable):
    # If you are querying an entire table then mark entireTable=True
    # If you are querying at the variable level then mark entireTable=False
    # Define dictionary of tableCodes and tableTypes 
    tableDict = {
        "D":"/profile",
        "S":"/subject",
        "B":""
    }
    # Find the tableTypes the user is requesting
    tableList = set([variables_toquery[i][0] for i in range(len(variables_toquery))])

    # If the query is for specific variables, generate the dictionary this way:
    if entireTable:
        # Find the variables that fall under each tableCode
        tableCodes = {
            x:{
                # Get the tableType from the tableDict
                "tableType":tableDict[x], 
                # Join the tableVariables together in a string
                "tableVariables":{
                    # Get the list of tableVariables for that tableType
                    ("group(" + variables_toquery[i] + ")") for i in range(len(variables_toquery)) if variables_toquery[i][0]==x
                }
            } for x in tableList
        }

    # If the query is for specific variables, generate the dictionary this way:
    else:
        # Find the variables that fall under each tableCode
        tableCodes = {
            x:{
                # Get the tableType from the tableDict
                "tableType":tableDict[x], 
                # Join the tableVariables together in a string
                "tableVariables":"GEO_ID," + "NAME," + ",".join({
                    # Get the list of tableVariables for that tableType
                    variables_toquery[i] for i in range(len(variables_toquery)) if variables_toquery[i][0]==x
                })
            } for x in tableList
        }

    return tableCodes

In [33]:
def parseCensus(url):
    # Use BeautifulSoup to parse html and find the document text
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    censusData = soup.get_text()
    
    # Clean data by removing brackets
    censusData = censusData.replace("[", "")
    censusData = censusData.replace("]", "")

    # Create dataframe from string data
    censusData = pd.read_csv(StringIO(censusData), sep=",")
    return censusData

In [34]:
def fetchCensusTracts(year, state, county, tractList, variables_toquery, entireTable, apiKey):
    # If you are querying an entire table then mark entireTable=True
    # If you are querying at the variable level then mark entireTable=False
    # Create empty df to hold the data
    censusData = pd.DataFrame(columns=['GEO_ID', 'NAME', 'state', 'county', 'tract'])
    # Generate dictionary of table codes, table types, and census variables
    tableCodes = buildTableCodes(variables_toquery, entireTable)
    # Check if the query is for full tables
    if entireTable:
        for code in tableCodes.values():
            # Find the tableType and tableVariable for that type of table
            tableType = code["tableType"]
            # Loop over the tables because the census API will not let you query multiple tables
            for tableVariables in code["tableVariables"]:
                # Build a url specific to each tableType
                url = f'https://api.census.gov/data/{year}/acs/acs5{tableType}?key={apiKey}&in=state:{state}%20county:{county}&for=tract:{tractList}&get={tableVariables}'
                # Parse the data returned from the census API
                tableData = parseCensus(url)    
                # Drop final column which is empty
                tableData = tableData.drop(tableData.columns[-1], axis=1)
                # Merge each iteration of tableData into the full censusData
                censusData = censusData.merge(tableData, how="outer", on=['GEO_ID', 'NAME', 'state', 'county', 'tract'])
    # If the query is for individual variables
    else:
        # Loop over the types of tables
        for code in tableCodes.values():
            # Find the tableType and tableVariable for that type of table
            tableType = code["tableType"]
            tableVariables = code["tableVariables"]
            # Build a url specific to each tableType
            url = f'https://api.census.gov/data/{year}/acs/acs5{tableType}?key={apiKey}&in=state:{state}%20county:{county}&for=tract:{tractList}&get={tableVariables}'
            # Parse the data returned from the census API
            tableData = parseCensus(url)    
            # Drop final column which is empty
            tableData = tableData.drop(tableData.columns[-1], axis=1)
            # Merge each iteration of tableData into the full censusData
            censusData = censusData.merge(tableData, how="outer", on=['GEO_ID', 'NAME', 'state', 'county', 'tract'])
    # Reorder the columns to make it easier to read
    censusData.insert(0, 'tract', censusData.pop('tract'))
    censusData.insert(0, 'county', censusData.pop('county'))
    censusData.insert(0, 'state', censusData.pop('state'))
    censusData.insert(0, 'NAME', censusData.pop('NAME'))
    censusData.insert(0, 'GEO_ID', censusData.pop('GEO_ID'))

    # Clean the GEO_ID column so it matches census shapefiles
    censusData['GEO_ID'] = censusData['GEO_ID'].astype('string')
    censusData['GEO_ID'] = censusData['GEO_ID'].str.replace("1400000US", "")
    return censusData

In [36]:
def fetchCensusZips(year, zipList, variables_toquery, entireTable, apiKey):
    # If you are querying an entire table then mark entireTable=True
    # If you are querying at the variable level then mark entireTable=False
    # Create empty df to hold the data
    censusData = pd.DataFrame(columns=['GEO_ID', 'NAME', 'zip code tabulation area'])
    # Generate dictionary of table codes, table types, and census variables
    tableCodes = buildTableCodes(variables_toquery, entireTable)
    # Check if the query is for full tables
    if entireTable:
        for code in tableCodes.values():
            # Find the tableType and tableVariable for that type of table
            tableType = code["tableType"]
            # Loop over the tables because the census API will not let you query multiple tables
            for tableVariables in code["tableVariables"]:
                # Build a url specific to each tableType
                url = f'https://api.census.gov/data/{year}/acs/acs5{tableType}?key={apiKey}&for=zip%20code%20tabulation%20area:{zipList}&get={tableVariables}'
                # Parse the data returned from the census API
                tableData = parseCensus(url)    
                # Drop final column which is empty
                tableData = tableData.drop(tableData.columns[-1], axis=1)
                # Merge each iteration of tableData into the full censusData
                censusData = censusData.merge(tableData, how="outer", on=['GEO_ID', 'NAME', 'zip code tabulation area'])
    # If the query is for individual variables
    else:
        # Loop over the types of tables
        for code in tableCodes.values():
            # Find the tableType and tableVariable for that type of table
            tableType = code["tableType"]
            tableVariables = code["tableVariables"]
            # Build a url specific to each tableType
            url = f'https://api.census.gov/data/{year}/acs/acs5{tableType}?key={apiKey}&for=zip%20code%20tabulation%20area:{zipList}&get={tableVariables}'
            # Parse the data returned from the census API
            tableData = parseCensus(url)    
            # Drop final column which is empty
            tableData = tableData.drop(tableData.columns[-1], axis=1)
            # Merge each iteration of tableData into the full censusData
            censusData = censusData.merge(tableData, how="outer", on=['GEO_ID', 'NAME', 'zip code tabulation area'])
    # Reorder the columns to make it easier to read
    censusData.insert(0, 'zip code tabulation area', censusData.pop('zip code tabulation area'))
    censusData.insert(0, 'NAME', censusData.pop('NAME'))
    censusData.insert(0, 'GEO_ID', censusData.pop('GEO_ID'))

    # Clean the GEO_ID column so it matches census shapefiles
    censusData['GEO_ID'] = censusData['GEO_ID'].astype('string')
    censusData['GEO_ID'] = censusData['GEO_ID'].str.replace("8600000US", "")

    # Limit to texas zip codes
    censusData = censusData.loc[(censusData['zip code tabulation area']>=77000) & (censusData['zip code tabulation area']<78000)]
    return censusData

# Test functions

In [38]:
### Select data to query
apiKey = MY_API_KEY
# Select year
year = CURRENT_YEAR
# Select geography
state = TEXAS
county = FORTBEND
zipList = FtBendZips
# Select census variables
entireTable = False
variables_toquery = [
    'DP05_0005E','DP05_0001E','DP03_0062E',
    'S1701_C02_001E','S1701_C01_001E',
    'B14006_004E','B14006_005E','B14006_006E',
    'B17020_001E','B17020_003E', 'B17020_004E'
]

DataRequest_ECE_Zips = fetchCensusZips(year, zipList, variables_toquery, entireTable, apiKey)
DataRequest_ECE_Zips.head()

,GEO_ID,NAME,zip code tabulation area,DP05_0001E,DP05_0005E,DP03_0062E,S1701_C02_001E,S1701_C01_001E,B17020_004E,B14006_004E,B17020_001E,B14006_006E,B14006_005E,B17020_003E
0,77583,ZCTA5 77583,77583,41305,3160,89406,2961,35134,452,0,35134,266,66,198
1,77464,ZCTA5 77464,77464,70,0,-666666666,7,70,0,0,70,0,0,0
2,77493,ZCTA5 77493,77493,40547,3072,91670,2434,40416,285,20,40416,243,17,172
3,77494,ZCTA5 77494,77494,126310,9498,142037,5845,125973,650,419,125973,555,58,964
4,77435,ZCTA5 77435,77435,3857,136,59079,337,3857,59,3,3857,33,24,27


In [39]:
### Select data to query
apiKey = MY_API_KEY
# Select year
year = CURRENT_YEAR
# Select geography
state = TEXAS
county = FORTBEND
tractList = ALL
# Select census variables
entireTable = False
variables_toquery = [
    'DP05_0005E','DP05_0001E','DP03_0062E',
    'S1701_C02_001E','S1701_C01_001E',
    'B14006_004E','B14006_005E','B14006_006E',
    'B17020_001E','B17020_003E', 'B17020_004E'
]

DataRequest_ECE_Tracts = fetchCensusTracts(year, state, county, tractList, variables_toquery, entireTable, apiKey)
DataRequest_ECE_Tracts.head()

,GEO_ID,NAME,state,county,tract,DP05_0001E,DP05_0005E,DP03_0062E,S1701_C02_001E,S1701_C01_001E,B17020_004E,B14006_004E,B17020_001E,B14006_006E,B14006_005E,B17020_003E
0,48157670101,"Census Tract 6701.01, Fort Bend County, Texas",48,157,670101,6436,347,53971,1094,6416,109,62,6416,109,38,165
1,48157670102,"Census Tract 6701.02, Fort Bend County, Texas",48,157,670102,3377,291,51216,674,3377,107,0,3377,93,46,170
2,48157670201,"Census Tract 6702.01, Fort Bend County, Texas",48,157,670201,2153,159,93000,269,2073,35,0,2073,0,0,0
3,48157670202,"Census Tract 6702.02, Fort Bend County, Texas",48,157,670202,5389,416,49500,714,5389,188,0,5389,145,7,83
4,48157670300,"Census Tract 6703, Fort Bend County, Texas",48,157,670300,3446,142,57325,673,3446,86,0,3446,46,0,48


In [41]:
### Select data to query
apiKey = MY_API_KEY
# Select year
year = CURRENT_YEAR
# Select geography
state = TEXAS
county = FORTBEND
tractList = ALL
# Select census variables
entireTable = True
variables_toquery = [
    'DP05', 'DP03'
]

fullTables = fetchCensusTracts(year, state, county, tractList, variables_toquery, entireTable, apiKey)
fullTables.head()

C:\Users\tharry\AppData\Local\Temp\ipykernel_10252\418174184.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  censusData.insert(0, 'tract', censusData.pop('tract'))
C:\Users\tharry\AppData\Local\Temp\ipykernel_10252\418174184.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  censusData.insert(0, 'county', censusData.pop('county'))
C:\Users\tharry\AppData\Local\Temp\ipykernel_10252\418174184.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which ha

,GEO_ID,NAME,state,county,tract,DP05_0001E,DP05_0001EA,DP05_0001M,DP05_0001MA,DP05_0001PE,...,DP03_0136PM,DP03_0136PMA,DP03_0137E,DP03_0137EA,DP03_0137M,DP03_0137MA,DP03_0137PE,DP03_0137PEA,DP03_0137PM,DP03_0137PMA
0,48157670101,"Census Tract 6701.01, Fort Bend County, Texas",48,157,670101,6436,NaN,820,NaN,6436,...,7.8,NaN,-888888888,(X),-888888888,(X),20.3,NaN,10.8,NaN
1,48157670102,"Census Tract 6701.02, Fort Bend County, Texas",48,157,670102,3377,NaN,497,NaN,3377,...,8.9,NaN,-888888888,(X),-888888888,(X),30.0,NaN,20.3,NaN
2,48157670201,"Census Tract 6702.01, Fort Bend County, Texas",48,157,670201,2153,NaN,741,NaN,2153,...,15.8,NaN,-888888888,(X),-888888888,(X),33.5,NaN,30.3,NaN
3,48157670202,"Census Tract 6702.02, Fort Bend County, Texas",48,157,670202,5389,NaN,776,NaN,5389,...,9.4,NaN,-888888888,(X),-888888888,(X),5.0,NaN,8.3,NaN
4,48157670300,"Census Tract 6703, Fort Bend County, Texas",48,157,670300,3446,NaN,639,NaN,3446,...,12.0,NaN,-888888888,(X),-888888888,(X),25.5,NaN,15.6,NaN


# Print to csv

In [32]:
DataRequest_ECE.to_csv("data/Output.csv")